In [1]:
data = """
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
"""
with open("input_18.txt", 'r') as f:
    data = f.read()

data = [x.split(" ") for x in data.split("\n") if not x == ""]

#data= data.replace("\n", "")

In [2]:
data

[['set', 'i', '31'],
 ['set', 'a', '1'],
 ['mul', 'p', '17'],
 ['jgz', 'p', 'p'],
 ['mul', 'a', '2'],
 ['add', 'i', '-1'],
 ['jgz', 'i', '-2'],
 ['add', 'a', '-1'],
 ['set', 'i', '127'],
 ['set', 'p', '826'],
 ['mul', 'p', '8505'],
 ['mod', 'p', 'a'],
 ['mul', 'p', '129749'],
 ['add', 'p', '12345'],
 ['mod', 'p', 'a'],
 ['set', 'b', 'p'],
 ['mod', 'b', '10000'],
 ['snd', 'b'],
 ['add', 'i', '-1'],
 ['jgz', 'i', '-9'],
 ['jgz', 'a', '3'],
 ['rcv', 'b'],
 ['jgz', 'b', '-1'],
 ['set', 'f', '0'],
 ['set', 'i', '126'],
 ['rcv', 'a'],
 ['rcv', 'b'],
 ['set', 'p', 'a'],
 ['mul', 'p', '-1'],
 ['add', 'p', 'b'],
 ['jgz', 'p', '4'],
 ['snd', 'a'],
 ['set', 'a', 'b'],
 ['jgz', '1', '3'],
 ['snd', 'b'],
 ['set', 'f', '1'],
 ['add', 'i', '-1'],
 ['jgz', 'i', '-11'],
 ['snd', 'a'],
 ['jgz', 'f', '-16'],
 ['jgz', 'a', '-19']]

In [3]:
import sys
def machine(pid):
    numSend = 0
    #print(f"hello {pid}")
    #sys.stdout.flush()
    registers = {"p": pid}
    ptr = 0
    while ptr < len(data):    
        op, X, *Y = data[ptr]
        Y = Y or None
        if Y:
            Y = Y[0]
            if Y.isalpha():
                Y = registers[Y] #getting register value
            Y = int(Y)
        for x in [x for x in [X, Y] if x]:
            if str(x).isalpha() and not x in registers.keys():
                registers[x] = 0

        if op == "snd":
            #send to other program
            if X in registers.keys():
                X = registers[X]
            cv.acquire()
            #print(f"sending {X}")
            box = sharedBox[(pid+1)%2]
            box.append(X)
            sharedBox[(pid+1)%2] = box
            if pid == 1:
                numSend += 1
            cv.notify()
            cv.release()
        elif op == "set":
            #set value X = val(Y)
            registers[X] = Y  
        elif op == "add":
            #add value X:= val(X) + val(Y)
            registers[X] = registers[X] + Y 
        elif op == "mul":
            #multiply value X:= val(X) * val(Y)
            registers[X] = registers[X] * Y
        elif op == "mod":
            #modulo value X:= val(X) % val(Y)
            registers[X] = registers[X] % Y
        elif op == "rcv":
            #receive from box
            cv.acquire()
            box = sharedBox[pid]
            if len(box) == 0:
                if pid == 1:
                    print(numSend)
                cv.wait()
                box = sharedBox[pid]
            registers[X] = box.pop(0)
            #print(f"received {registers[X]}")
            sharedBox[pid] = box
            cv.release()
        elif op == "jgz":
            #jump
            if X in registers.keys():
                X = registers[X]
            if int(X) > 0:
                ptr += Y
                continue
        ptr += 1

In [ ]:
from multiprocessing import Process
from threading import Thread, Lock, Condition
sharedBox = {0: [], 1:[]}
#mutex = Lock()
cv = Condition()
p0 = Thread(target=machine, args = (0,))
p0.start()
p1 = Thread(target=machine, args = (1,))
p1.start()
p0.join()
p1.join()

127
254
381
508
635
762
889
1016
1143
1270
1397
1524
1651
1778
1905
2032
2159
2214
2339
2464
2589
2677
2802
2927
3052
3098
3223
3348
3473
3560
3685
3810
3937
3975
4100
4225
4350
4419
4544
4669
4794
4919
5044
5169
5356
5481
5606
5731
5856
5981
6106
6231
6356
6481
6606
6731
6858
6985
7112
7308
7433
7558
8001
8001


In [ ]:
t = [0,1,2,3]
t.pop(0)

In [2]:
t

[1, 2, 3]